<a href="https://colab.research.google.com/github/ehdob-cid20/trabalho-pratico/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Pré Processamento

## Importando as bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Importando o *dataset*

### Verifica se o dataset já foi importado para o google drive

Um script que verifica se o dataset já foi importado para o google drive. Caso ainda não tenha sido importado, ele faz o download do arquivo e extrai para a pasta informada na variável **dirName**.

Variáveis:

> **datasetFileName** é o nome do arquivo com extensão.

> **datsetUrl** é o endereço web onde o arquivo está disponível.

> **dirName** é o diretório do google drive onde o dataset será salvo.

Nota: O diretório de salvamento (*dirName*) precisa existir.


In [6]:
import os

datasetFileName = 'dados-curso-completo.csv.tar.gz'
datsetUrl = 'http://tiagodemelo.info/datasets/dados-curso-completo.csv.tar.gz'
dirName = 'trabalho-pratico' # Este diretorio pode ser alterado se ja tiver o dataset no google drive.

if False == os.path.isfile('/content/drive/My Drive/' + dirName + '/' + datasetFileName[:-7]):
  if False == os.path.isdir('/content/drive'):
    print('Google driver não está montado!')
  elif False == os.path.isdir('/content/drive/My Drive/' +  dirName + '/'):
    print('Diretório ' + dirName + ' não encontrado.')
  elif False == os.path.isfile('/content/drive/My Drive/' + dirName + '/' + datasetFileName):
    !wget -O '/content/drive/My Drive/$dirName/$datasetFileName' '$datsetUrl' #TODO: handle download errors.
    !tar -C '/content/drive/My Drive/$dirName/{datasetFileName[:-7]}' -zxf '/content/drive/My Drive/$dirName/$datasetFileName' #TODO: handle extract errors.
  else:
    !tar -C '/content/drive/My Drive/$dirName/' -zxf '/content/drive/My Drive/$dirName/$datasetFileName' #TODO: handle extract errors.
else:
  print('O dataset ' + datasetFileName[:-7] + ' foi encontrado.')

O dataset dados-curso-completo.csv foi encontrado.


### Importando o *dataset* usando pandas

In [7]:
dataset = pd.read_csv('/content/drive/My Drive/' + dirName + '/' + datasetFileName[:-7])

## Visualizando o *dataset* inicial (sem qualquer tratamento)

## Aplicando tratamento ao *dataset*

### Removendo duplicatas

### Categorizando variáveis

## Dividindo o dataset em *Training set* e *Test set*